In [1]:
import tensorflow as tf
import os
import random
import sys
from PIL import Image
import numpy as np

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
_NUM_TEST = 500
_RANDOM_SEED = 0
DATASET_DIR = 'captcha/images/'
TFRECORD_DIR = 'captcha/'

In [3]:
def _dataset_exists(dataset_dir):
    for split_name in ['trian','test']:
        output_filename = os.path.join(dataset_dir,split_name+ '.tfrecords')
        if not tf.gfile.Exists(output_filename):
            return False
        return True
    
def _get_filename_and_classes(dataset_dir):
    photo_filenames = []
    for filename in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filename)
        photo_filenames.append(path)
    return photo_filenames

def int64_feature(values):
    if not isinstance(values,(tuple,list)):
        values = [values]
    return tf.train.Feature(int64_list = tf.train.Int64List(value = values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [values])) 

def image_to_tfexample(image_data, label0, label1, label2, label3):
    #Abstract base class for protocal messages
    return tf.train.Example(features = tf.train.Features(feature = {
        'image': bytes_feature(image_data),
        'label0': int64_feature(label0),
        'label1': int64_feature(label0),
        'label2': int64_feature(label0),
        'label3': int64_feature(label0),
    }))


def _convert_dataset(split_name, filenames, dataset_dir):
    assert split_name in ['train','test']

    with tf.Session() as sess:
        output_filename = os.path.join(TFRECORD_DIR, split_name + '.tfrecords')
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            for i, filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d / %d' % (i+1, len(filenames)))
                    sys.stdout.flush()
                    
                    # image
                    image_data = Image.open(filename)                    
                    image_data = image_data.resize((224,224))                    
                    image_data = np.array(image_data.convert('L'))  #convert to grayscale                    
                    image_data = image_data.tobytes()
                    # label
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        num_labels.append(int(labels[j]))
                    
                    example = image_to_tfexample(image_data, num_labels[0],num_labels[1],num_labels[2],num_labels[3])
                    tfrecord_writer.write(example.SerializeToString())

                except IOError as e:
                    print("Could not read:", filename)
                    print("Error:",e)
                    print("Skip it\n")
    sys.stdout.write('\n')
    sys.stdout.flush()

In [4]:
if _dataset_exists(TFRECORD_DIR):
    print("TFRecord file already exists!")
    
else:
    # get file names and split into train & test 
    photo_filenames = _get_filename_and_classes(DATASET_DIR)    
    random.seed(_RANDOM_SEED)
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[_NUM_TEST:]
    testing_filenames = photo_filenames[:_NUM_TEST]

    
    _convert_dataset('train',training_filenames, DATASET_DIR)
    _convert_dataset('test', testing_filenames, DATASET_DIR)
    print("Finish generating TFRecord")

>> Converting image 5822 / 5822
>> Converting image 500 / 500
Finish generating TFRecord
